## This HW depicts various models for training the provided Dataset - Rubab Karim

### Importing the Winequaity-red Dataset in order to conduct an analysis of the data

In [12]:
import torch
import imageio
import os

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

from torchvision import datasets
from torchvision import transforms

import sklearn
from collections import Counter

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, accuracy_score, f1_score


from sklearn.preprocessing import LabelEncoder

In [18]:
#wine_path = 'data/winequality-red.csv'
#wineq_np = np.loadtxt(wine_path, dtype = np.float32, delimiter = ';', skiprows = 0)

wineq = pd.read_csv('data/winequality-red.csv', header = None, skiprows = 0)
# wineq_np = 

# t_c = torch.from_numpy(  wineq_np  )

# Reading the 'data/winequality-white.csv' file
# white = pd.read_csv("data/winequality-white.csv", sep=';')

# X = t_c[:, :-1]
# Y = t_c[:, -1]

X = wineq.loc[1:, :3].values
X = X.astype(float)


y = wineq.loc[1:,  4 ].values

print(X.shape)
print(y.shape)

le = LabelEncoder()
y = le.fit_transform(   y   )

(1599, 4)
(1599,)


In [46]:
# X = torch.tensor(  X   )
# Y= torch.tensor(  y   )

# print(t_u.shape)
# print(t_c.shape)

t_c = X.unsqueeze(1)
t_u = Y.unsqueeze(1)

print(t_u.shape)
print(t_c.shape)

torch.Size([1599, 1])
torch.Size([1599, 1, 4])


## After pre-processing, we split the dataset

In [47]:
n_samples = t_u.shape[0]
n_samples

1599

In [48]:
n_val = int(0.3 * n_samples)
n_val

479

In [49]:
shuffled_indeces = torch.randperm(n_samples)
shuffled_indeces

tensor([ 704, 1160,  391,  ...,  443, 1345, 1190])

In [50]:
train_indeces = shuffled_indeces[:-n_val]
val_indeces = shuffled_indeces[-n_val:]

print(train_indeces)
print(val_indeces)

tensor([ 704, 1160,  391,  ...,  621, 1454,  329])
tensor([1084,  191, 1001, 1290, 1373,  737, 1173, 1241, 1346, 1333,  416, 1006,
         612,  214, 1093,  176,  888,  477, 1033, 1050,  163,  545,  415,  945,
        1030,   65,  961,  619,  440,  753,  483,  114,  422,  573,  351, 1368,
          94, 1066, 1166,  921,    1,  883,  554, 1415,  267, 1551,  495,  782,
        1500,   89,   79,  112,  825,  219, 1498, 1165,  514, 1007,  632,  205,
         424, 1014, 1387,    0, 1420,  236, 1239,  131,  878, 1163,  983,  284,
         542,  858,  360,  994, 1269, 1353,  462,  723,  279,  591, 1242, 1307,
        1582,  529, 1300, 1565,  299,  563, 1528, 1480,  204,  770,  696,  868,
         130, 1288,  693,  788,  605, 1240,  775, 1383, 1370, 1265,  394,  785,
         640, 1470,  192, 1453,  610, 1216, 1275, 1276,  269,  446,  100, 1475,
        1123, 1057,  103,  347,   80,  801, 1095,   98,  643,  173,  874,  496,
        1594,  540,  187, 1581,  372,  988,  199,  381,  584,  533, 1

In [51]:
train_t_u = t_u[train_indeces]
train_t_c = t_c[train_indeces]

val_t_u = t_u[val_indeces]
val_t_c = t_c[val_indeces]

train_t_u = 0.1 * train_t_u
val_t_u   = 0.1 * val_t_u

## After prepping the dataset, we now push it into the Training Loop

In [52]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_t_u, val_t_u, train_t_c, val_t_c):
    
    for epoch in range(1, n_epochs):
        
        train_t_p = model(train_t_u)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        
        if epoch % 50 == 0:
            print("Train Loss: ", train_loss.item())
            print("Val Loss: ", val_loss.item())

### Here, we define the Regression Models that are going to be used in this analysis. 

#### a) Linear Regression Model

In [53]:
def model_fn_linear():
    return nn.Linear(1, 1)

#### b) Multi-Layer Perceptron

In [54]:
def model_fn_nn_1hidden():
    
    seq_model = nn.Sequential(
        nn.Linear(1599, 599),
        nn.ReLU(),
        
        nn.Linear(599, 1),
    
    )
    
   # optimizer = optim.SGD(
   #    model_fn_nn_1hidden.parameters(),
   #    lr=1e-3  )
    
    return seq_model

#### c) Deep Neural Network with 2 hidden layers

In [55]:
def model_deep_learning_2hidden():
    seq_model = nn.Sequential(
        nn.Linear(1599, 1099),
        nn.ReLU(),
        
        nn.Linear(1099, 599),
        nn.Tanh(),
        
        nn.Linear(599, 1)
    )
    
    return seq_model

#### d) Deep Neural Network with 4 hidden layers

In [56]:
def model_deep_learning_4hidden():
    seq_model = nn.Sequential(
        nn.Linear(1599, 1099),
        nn.ReLU(),
        
        nn.Linear(1099, 599),
        nn.Tanh(),
        
        nn.Linear(599, 199),
        nn.Tanh(),
        
        nn.Linear(199, 99),
        nn.Tanh(),
        
        nn.Linear(99, 1)
    )
    
    return seq_model

### Now, we call the function loop and push data into the training loop

In [57]:
#model = model_fn_linear()

model = model_fn_nn_1hidden()

#model = model_deep_learning_2hidden()

#model = model_deep_learning_4hidden()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.MSELoss().float()

In [58]:
result = training_loop(
    
    n_epochs = 1000,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_t_u = train_t_u,
    val_t_u = val_t_u,
    train_t_c = train_t_c,
    val_t_c = val_t_c
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1120x1 and 1599x599)

## 